In [ ]:
import geemap
import os
import ee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
out_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

MODIS = ee.ImageCollection('MODIS/006/MCD12Q1')

### Check first image of collection

In [ ]:
first_image = MODIS.first()

vis = {
    'bands': ['LC_Type1'],
    'min': 1.0,
    'max': 17.0,
    'palette': ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'],}

Map.addLayer(first_image, vis, 'First image')
Map.add_legend(builtin_legend='MODIS/051/MCD12Q1')

### Create collection with specific years

In [ ]:
MODIS_layers = MODIS.aggregate_array('system:id').getInfo()
print(MODIS_layers)

In [ ]:
MODIS2001 = ee.Image('MODIS/006/MCD12Q1/2001_01_01').select('LC_Type1')
MODIS2003 = ee.Image('MODIS/006/MCD12Q1/2003_01_01').select('LC_Type1')
MODIS2005 = ee.Image('MODIS/006/MCD12Q1/2005_01_01').select('LC_Type1')
MODIS2007 = ee.Image('MODIS/006/MCD12Q1/2007_01_01').select('LC_Type1')
MODIS2009 = ee.Image('MODIS/006/MCD12Q1/2009_01_01').select('LC_Type1')
MODIS2011 = ee.Image('MODIS/006/MCD12Q1/2011_01_01').select('LC_Type1')
MODIS2013 = ee.Image('MODIS/006/MCD12Q1/2013_01_01').select('LC_Type1')
MODIS2015 = ee.Image('MODIS/006/MCD12Q1/2015_01_01').select('LC_Type1')
MODIS2017 = ee.Image('MODIS/006/MCD12Q1/2017_01_01').select('LC_Type1')

MODIS2 = ee.ImageCollection([MODIS2001, MODIS2003, MODIS2005, MODIS2007, MODIS2009, MODIS2011, MODIS2013, MODIS2015, MODIS2017])

MODIS2_layer_names = ['MODIS2 ' + str(year) for year in range(2001, 2019, 2)]
print(MODIS2_layer_names)
print(MODIS2.size().getInfo())

### Center the map to the region you want to check

In [ ]:
Map3 = geemap.Map(center=[15.066759142205628, 106.84422868614095], zoom=8)
Map3

### You can select an area in the map and use the draw_last_feature to define roi

In [ ]:
region = Map3.draw_last_feature
roi=region.geometry()
print(roi.getInfo())

### Or just select a geometry

In [ ]:
roi = ee.Geometry.Polygon(
    [[[106.269001, 14.478194], [106.269001, 15.515808], [107.309188, 15.515808], 
      [107.309188, 14.478194], [106.269001, 14.478194]]], None, False)

### Select the band from MODIS and the frames per second

In [ ]:
video_args = {
  'dimensions': 768,
  'region': roi,
  'framesPerSecond': 1,
  'bands': ['LC_Type1'],
  'min': 1.0,
  'max': 17.0,
  'palette': ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff']
}

In [ ]:
work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, "landcover_ts.gif")

In [ ]:
geemap.download_ee_video(MODIS2, video_args, out_gif)

In [ ]:
geemap.show_image(out_gif)

### Add the labels

In [ ]:
texted_gif = os.path.join(work_dir, "landcover_ts_text.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('70%', '5%'), text_sequence=MODIS2_layer_names, font_size=30, font_color='#000000', add_progress_bar=True,duration=1000)

In [ ]:
label = 'Natural Degradation South Laos'
geemap.add_text_to_gif(texted_gif, texted_gif, xy=('15%', '88%'), text_sequence=label, font_size=30, font_color='#000000', progress_bar_color='red',duration=1000)

In [ ]:
geemap.show_image(texted_gif)